# Rat LPS Image Preprocessing

19 May 2022

Objective: Preprocess the given unlabeled images in `.npy` to labeled images in `.tif` for further VAMPIRE analysis.

## Read, label, and write images

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import skimage
import vampire as vp

In [ ]:
# set up filepaths to ndarrays
DATA_DIR = r'C:\Files\github-projects\nance-lab-data\microfiber\0-raw-rat-lps-ostu'
filenames = np.array(os.listdir(DATA_DIR))
os_path_join = np.vectorize(os.path.join)
filepaths = os_path_join(DATA_DIR, filenames)

In [ ]:
def load_imgs(filepaths):
    """Filepaths are paths to images. Returns image ndarrays."""
    imgs = []
    for filepath in filepaths:
        imgs.append(skimage.measure.label(np.load(filepath)))
    return imgs

In [ ]:
imgs = load_imgs(filepaths)

In [ ]:
img_paths = pd.Series(filepaths).str.replace('npy', 'tif').to_numpy()

In [ ]:
from skimage import io
def save_imgs(img_set, save_img_paths):
    for i in range(len(img_set)):
        io.imsave(save_img_paths[i], img_set[i])

In [ ]:
save_imgs(imgs, img_paths)

Below are not runned for rat LPS model

In [ ]:
import pandas as pd
import numpy as np
region = 'thalamus'
filter_info = pd.DataFrame(np.array([['tif'], ['otsu'], [region]]).T,
                   columns=['extension', 'threshold', 'region'])

import vampire as vp
vp.model.build_and_apply_model(r'C:\Files\github-projects\nance-lab-data\microfiber\ogd_severity_segmentations',
                      r'C:\Files\github-projects\nance-lab-data\microfiber\vampire-data',
                      f'otsu_contour_split_{region}',
                      f'otsu_contour_split_{region}',
                      50, 5, filter_info,
                      random_state=1)

## Contour test train split

In [ ]:
def extract_contours(img_set):
    img_i = 1
    contours_from_folder = []
    for img in img_set:
        # read image and get contours and properties
        object_labels = np.unique(img)[1:]
        contours_from_img = vp.extraction.get_contours_from_image(img, object_labels)
        contours_from_folder.extend(contours_from_img)
        img_i += 1
    return contours_from_folder

In [ ]:
otsu_contours = extract_contours(otsu_imgs)
yen_contours = extract_contours(yen_imgs)

In [ ]:
def extract_properties(img_set, filepaths):
    properties_from_folder = []
    for i in range(len(img_set)):
        # read image and get contours and properties
        object_labels = np.unique(img_set[i])[1:]
        properties_from_img = vp.extraction.get_properties_from_image(img_set[i], filepaths[i], i, object_labels)
        properties_from_folder.append(properties_from_img)
    properties_from_folder_df = pd.concat(properties_from_folder)
    return properties_from_folder_df

In [ ]:
otsu_properties = extract_properties(otsu_imgs, otsu_filepaths)
yen_properties = extract_properties(yen_imgs, yen_filepaths)

### Otsu

In [ ]:
from sklearn.model_selection import train_test_split
otsu_contours_train, otsu_contours_test, otsu_properties_train, otsu_properties_test = train_test_split(otsu_contours, otsu_properties, random_state=1)

In [ ]:
otsu_model = vp.model.build_model_from_contours(otsu_contours_train, 'Otsu OGD', 50, 5, random_state=1)
otsu_contours_df, otsu_min_distance = vp.model.apply_model_from_contours(otsu_contours_test, otsu_model)

In [ ]:
def write_clusters_info(properties_df, contours_df, distance):
    properties_df['cluster_id'] = contours_df['cluster_id']
    properties_df['distance_to_centroid'] = distance
    return properties_df

In [ ]:
write_clusters_info(otsu_properties_test, otsu_contours_df, otsu_min_distance)

In [ ]:
output_path = r'C:\Files\github-projects\nance-lab-data\microfiber\figure'
vp.plot.set_plot_style()
vp.plot.plot_distribution_contour_dendrogram(otsu_model['build_contours_df'], None, output_path, otsu_model['model_name'])
vp.plot.plot_distribution_contour_dendrogram(otsu_model['build_contours_df'], otsu_contours_df, output_path,
                                                  otsu_model['model_name'], otsu_model['model_name'])

### Yen

In [ ]:
from sklearn.model_selection import train_test_split
yen_contours_train, yen_contours_test = train_test_split(yen_contours, random_state=1)

In [ ]:
yen_model = vp.model.build_model_from_contours(yen_contours_train, 'Yen OGD', 50, 5, random_state=1)
yen_contours_df, yen_min_distance = vp.model.apply_model_from_contours(yen_contours_test, yen_model)

In [ ]:
output_path = r'C:\Files\github-projects\nance-lab-data\microfiber\figure'
vp.plot.set_plot_style()
vp.plot.plot_distribution_contour_dendrogram(yen_model['build_contours_df'], None, output_path, yen_model['model_name'])
vp.plot.plot_distribution_contour_dendrogram(yen_model['build_contours_df'], yen_contours_df, output_path,
                                                  yen_model['model_name'], yen_model['model_name'])

## Original image split

In [ ]:
from sklearn.model_selection import train_test_split
otsu_imgs_train, otsu_imgs_test = train_test_split(otsu_imgs, random_state=1)
yen_imgs_train, yen_imgs_test = train_test_split(yen_imgs, random_state=1)

In [ ]:
def extract_contours(img_set):
    img_i = 1
    contours_from_folder = []
    for img in img_set:
        # read image and get contours and properties
        object_labels = np.unique(img)[1:]
        contours_from_img = vp.extraction.get_contours_from_image(img, object_labels)
        contours_from_folder.extend(contours_from_img)
        img_i += 1
    return contours_from_folder

### Otsu

In [ ]:
otsu_contours_train = extract_contours(otsu_imgs_train)
otsu_contours_test = extract_contours(otsu_imgs_test)

In [ ]:
otsu_model = vp.model.build_model_from_contours(otsu_contours_train, 'Otsu OGD', 50, 5, random_state=1)
otsu_contours_df, otsu_min_distance = vp.model.apply_model_from_contours(otsu_contours_test, otsu_model)

In [ ]:
output_path = r'C:\Files\github-projects\nance-lab-data\microfiber\figure'
vp.plot.set_plot_style()
vp.plot.plot_distribution_contour_dendrogram(otsu_model['build_contours_df'], None, output_path, otsu_model['model_name'])
vp.plot.plot_distribution_contour_dendrogram(otsu_model['build_contours_df'], otsu_contours_df, output_path,
                                                  otsu_model['model_name'], otsu_model['model_name'])

### Yen

In [ ]:
yen_contours_train = extract_contours(yen_imgs_train)
yen_contours_test = extract_contours(yen_imgs_test)

In [ ]:
yen_model = vp.model.build_model_from_contours(yen_contours_train, 'Yen OGD', 50, 5, random_state=1)
yen_contours_df, yen_min_distance = vp.model.apply_model_from_contours(yen_contours_test, yen_model)

In [ ]:
output_path = r'C:\Files\github-projects\nance-lab-data\microfiber\figure'
vp.plot.set_plot_style()
vp.plot.plot_distribution_contour_dendrogram(yen_model['build_contours_df'], None, output_path, yen_model['model_name'])
vp.plot.plot_distribution_contour_dendrogram(yen_model['build_contours_df'], yen_contours_df, output_path,
                                                  yen_model['model_name'], yen_model['model_name'])